In [1]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import os
import numpy as np
import openai
from serpapi import GoogleSearch
from dotenv import load_dotenv
import json

In [2]:
app = Flask(__name__)
CORS(app)

load_dotenv()

SERP_API_KEY = os.getenv('SERP_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
gpt_model = "gpt-4o"

In [5]:
def shorten_text(text, min, max):
    system_msg = "You are a helpful assistant that is good at shortening text."
    prompt = f"Shorten this text into {min} to {max} (inclusive) words: {text}"

    response = openai.ChatCompletion.create(
        model = gpt_model,
        api_key = OPENAI_API_KEY,
        messages = [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": prompt}
        ]
    )
    
    return response['choices'][0]['message']['content'].strip()

def google_shopping_search(query, num_results = 5):
    params = {
        "api_key": SERP_API_KEY,
        "engine": "google_shopping",
        "q": query,
        "hl": "en",
        "gl": "sg",
        "num": num_results
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    products = []

    for item in results.get("shopping_results", []):
        product = {
            "name": shorten_text(item.get("title"), 5, 8),
            "product_url": item.get("link"),
            "shop_name": shorten_text(item.get("source"), 1, 3),
            "img_url": item.get("thumbnail"),
            "price": item.get("price")
        }
        products.append(product)

    return products

In [6]:
print(google_shopping_search("blue t shirt"))

[{'name': 'Premium cotton soft tee, sapphire casual.', 'product_url': 'https://www.lazada.sg/products/premium-cotton-soft-style-tee-shirt-t-shirt-plain-sapphire-casual-business-i1172820747-s4650792699.html?from_gmc=1&fl_tag=1', 'shop_name': 'Lazada SG', 'img_url': 'https://encrypted-tbn0.gstatic.com/shopping?q=tbn:ANd9GcSPEL839QNn2VkIhE6heH0m25Xap9Ne_oObhoJSf-lS9CearoJjyDSCHb0BXtNwXJB4cjUrj0db5T08XcuuND0_bZno_54wfiXRwHmuquyCId5Oh7zgxmq8uQ&usqp=CAE', 'price': '$6.90'}, {'name': 'Navy Blue Gildan Cotton T-Shirt', 'product_url': 'https://shopee.sg/product/151878051/8119085500?gads_t_sig=VTJGc2RHVmtYMTlxTFVSVVRrdENkV1BNaHBLV2lHSzd2WEZjb2w0eUJubFJEajBnMlVMbC9wTFE0TTFGbkRqZGlUSXB5Tm5IcHNIVDVadUcva1ROL2JNMVZNdkpPWHBWWGFwNGg5ZklJREZVdlBvQ0I4NENQS21PdDIyZGZsUE0', 'shop_name': 'Online shopping', 'img_url': 'https://encrypted-tbn1.gstatic.com/shopping?q=tbn:ANd9GcSX7FPYb4eU8NqzkrrSXw6cZNDUnAz5KbjKo24MxizV3S3eWIn2tbss5mqnOZS0ar8v3NyazZIp9_OrLAMlbQR326S5trY94uKlBQXaknwcUdQRxWrjyJcmbg&usqp=CAE', 'pr

In [7]:
# DUMMY DATA
chat_histories = [[{
    "timestamp": "10/04/2025, 11:33:21 AM",
    "message": "Hii I am going to travel to Bali next week, do you have any ideas regarding things that I should bring?"
},
{
    "timestamp": "10/04/2025, 11:33:22 AM",
    "message": "Hmm noted, I don't have sunglasses and hat, maybe I will search for it at Bali."
},
{
    "timestamp": "10/04/2025, 11:33:23 AM",
    "message": "Ohhh yeah beach towel! Will buy it later too once I reach Bali."
},
{
    "timestamp": "10/04/2025, 11:33:24 AM",
    "message": "Huh pattern? I like flower patterns."
}]
]
user_data = {
    "gender": "female",
    "age": "19",
    "favorite_color": "white",
    "favorite_animal": "bunny",
}

In [8]:
def sentiment_analysis_with_keyword(chat_histories, user_data, keyword):
    system_msg = "You are a helpful assistant that is good at recommending gift for a person. Based on user's chat history, preference, and the keyword user entered, you must generate a very detailed and effective keyword to be used in serpapi google shopping search api."
    prompt = f"Chat history: {chat_histories}\nUser preference: {user_data}\nKeyword: {keyword}"

    response = openai.ChatCompletion.create(
        model = gpt_model,
        api_key = OPENAI_API_KEY,
        messages = [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": prompt}
        ]
    )
    
    return response['choices'][0]['message']['content'].strip()

In [9]:
print(sentiment_analysis_with_keyword(chat_histories, user_data, "hat"))

"women's flower pattern sun hat white beach travel accessory"


In [26]:
def generate_recommendation_with_keyword(chat_histories, user_data, keyword):
    search_keyword = sentiment_analysis_with_keyword(chat_histories, user_data, keyword)
    serpapi_list = google_shopping_search(search_keyword, 5)

    return serpapi_list

In [30]:
print(generate_recommendation_with_keyword(chat_histories, user_data, "hat"))

[{'name': "2020 Women's Big Wave Sunshade Beach Hat", 'product_url': 'https://shopee.sg/product/311337033/6853638221?gads_t_sig=VTJGc2RHVmtYMTlxTFVSVVRrdENkV1BNaHBLV2lHSzd2WEZjb2w0eUJubjJmMExKa2RtVkxhWTQxczh0cjNha2xwN2pHVTJtb1h5T3g1VldKbTRxME1qOG1BdEsvcTBKVWhDMStYdTlxa2VWMkVRQWxWZ2paYWtLWDEzbjFOSk0', 'shop_name': 'Online marketplace', 'img_url': 'https://encrypted-tbn2.gstatic.com/shopping?q=tbn:ANd9GcQ9XVKxO7jLBt55MwWKSY6mNZzTk0OlX-2t6i64vOZ5FHuUaWDGpYBAM0wMAfUq7MD2lrWwnKfrvvSJ_wHbLQV_ktTPqO8HJXd_0XEV8vk&usqp=CAE', 'price': '$7.12'}, {'name': 'Floral Crochet Detail Straw Hat', 'product_url': 'https://www.luulla.com/product/1829804/handwoven-straw-hat-with-floral-crochet-detail?b_currency=usd&set_b_currency=1&srsltid=AfmBOoqdpvIUsYQo7gAJLfLcXgGkXNwTHTLJPatMWLZR1OcVjY7MhgR2ckY', 'shop_name': 'Luulla Store', 'img_url': 'https://encrypted-tbn0.gstatic.com/shopping?q=tbn:ANd9GcQIkQbEQaTUMMNA2y3FqIbba4M-tsyQjny2XrYzUT6L393N6LU18s89KG1vLVDums-FErTpLRrAVr1rKTNURdIAp0nT5rRh7gV0xdaYDphNM8PbW8Ni

In [14]:
def sentiment_analysis_without_keyword(chat_histories, user_data):
    system_msg = "You are a helpful assistant that is good at recommending gifts for a person. Based on user's chat history and preference, you must generate 5 very detailed and effective keywords to be used in serpapi google shopping search api (separated by comma)."
    prompt = f"Chat history: {chat_histories}\nUser preference: {user_data}"

    response = openai.ChatCompletion.create(
        model = gpt_model,
        api_key = OPENAI_API_KEY,
        messages = [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": prompt}
        ]
    )
    
    raw_output = response['choices'][0]['message']['content'].strip()
    keyword_list = [kw.strip() for kw in raw_output.split(",")]
    return keyword_list

In [15]:
print(sentiment_analysis_without_keyword(chat_histories, user_data))

['flower pattern sunglasses', 'floral wide-brim hat', 'beach towel with floral design', 'white travel accessories', 'bunny-themed travel gear']


In [28]:
def generate_recommendation_without_keyword(chat_histories, user_data):
    search_keyword_list = sentiment_analysis_without_keyword(chat_histories, user_data)
    serpapi_list = []

    for search_keyword in search_keyword_list:
        serpapi_list.append(google_shopping_search(search_keyword, 1))

    return serpapi_list

In [29]:
print(generate_recommendation_without_keyword(chat_histories, user_data))

[[{'name': 'Tropical Floral Aviator Sunglasses', 'product_url': 'https://www.ebay.com/itm/224622594131?chn=ps&mkevt=1&mkcid=28&var=523440637840&google_free_listing_action=view_item&srsltid=AfmBOopUIGl-521xuOblHIwU29i4YkffdqM_HFyEJJkmaGmStG7joMy4bTU', 'shop_name': 'Online marketplace', 'img_url': 'https://encrypted-tbn3.gstatic.com/shopping?q=tbn:ANd9GcRKjjdvZCB-aJZ0RfawjvkEDGhMTxH_4tzqsc_Qe2KSvZIX5onMFYy0DalIB2ZBvxJ8Smv1sxKSbJbbyJ1k-mAT0HOLEIMiECKNhgIkbdN3x0xYUemcJzRC&usqp=CAE', 'price': '$11.54'}], [{'name': 'Wide brim UV protection visor sun hat', 'product_url': 'https://shopee.sg/product/25614013/2589667176?gads_t_sig=VTJGc2RHVmtYMTlxTFVSVVRrdENkYkVsYzY3b1A0bldEa1JSdzVkOFJGYVcza01zT0lKNVEvWHhXRHkzci91NnRjdTZoR2l3TnRzdGxqb3FBRTJVY1NUMUYrZTZ0UDlNKzNxazBVOXo5NW9yc2s1a3Z5WENoYTZFTzhncGdTTHg', 'shop_name': 'E-commerce platform', 'img_url': 'https://encrypted-tbn0.gstatic.com/shopping?q=tbn:ANd9GcQhPfGms_jD7JqDZMguDk-hu9jhHudCp57xe25_pWdSRMkWf-EmjvEF7aBo4cJArf8T2L-Ahy6i3zVpdKxTGteih4Ii0s_